In [18]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_predict

import pandas as pd 
#KLASSIFIKATION MIT SVM --> schlechtes Ergebnis --> LGBMClassifier file:classification_meth3 enthält die finale version

In [2]:
#read-in data
df_accidents = pd.read_csv('../../data/MA3_finished.csv', index_col='ID')

In [3]:
#verkleinern -> computional power reicht nur für frac der gesamten Daten aus
df_accidents = df_accidents.sample(n=350000, random_state=1)

In [4]:
df_accidents

,Unnamed: 0,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Description,Street,City,...,Nautical_Twilight,Astronomical_Twilight,Year,Month,Day,Hour,Duration,State Name,Region,Division
ID,,,,,,,,,,,,,,,,,,,,,
4290480,4290480,2,2022-10-11 13:36:37,2022-10-11 14:14:48,31.668610,-106.230205,0.881,Slow traffic on FM-1281 from Darrington Rd (FM...,Horizon Blvd,Horizon City,...,Day,Day,2022,10,1,13,2291.0,Texas,South,West South Central
6794167,6794167,2,2019-01-26 08:49:31,2019-01-26 09:19:14,41.933060,-86.572470,0.245,At Red Arrow Hwy/Exit 16 - Accident. Left lane...,I-94 W,Bridgman,...,Day,Day,2019,1,5,8,1783.0,Michigan,Midwest,East North Central
869970,869970,2,2021-06-23 09:58:28,2021-06-23 11:14:07,32.863430,-96.896896,0.000,Right lane blocked due to accident on TX-12-LO...,Technology Blvd E,Dallas,...,Day,Day,2021,6,2,9,4539.0,Texas,South,West South Central
5308122,5308122,2,2021-06-18 15:58:35,2021-06-18 17:17:09,29.684696,-95.557997,0.337,Slow traffic from Greenfork Dr to W Sam Housto...,W Sam Houston Pkwy S,Houston,...,Day,Day,2021,6,4,15,4714.0,Texas,South,West South Central
6608662,6608662,2,2020-02-18 18:51:08,2020-02-19 00:00:00,41.142360,-73.426950,0.000,At Main Ave/Exit 40 - Accident.,CT-15 N,Norwalk,...,Night,Day,2020,2,1,18,18532.0,Connecticut,Northeast,New England
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807425,1807425,2,2019-07-09 17:20:59,2019-07-09 19:24:42,30.231575,-97.717468,0.000,Accident on Riverside Dr at Crossing Pl.,E Riverside Dr,Austin,...,Day,Day,2019,7,1,17,7423.0,Texas,South,West South Central
4986977,4986977,2,2021-03-29 15:35:00,2021-03-29 17:14:14,37.052561,-121.253260,1.724,Incident on CA-152 EB near HOUSE 16242 Drive w...,Pacheco Pass Hwy,Hollister,...,Day,Day,2021,3,0,15,5954.0,California,West,Pacific
5817276,5817276,2,2021-01-05 13:41:30,2021-01-05 19:14:30,33.995029,-118.280350,1.110,Slow traffic on I-110 N - Harbor Fwy N from Sl...,W 52nd Pl,Los Angeles,...,Day,Day,2021,1,1,13,19980.0,California,West,Pacific


In [5]:
#featureselection
#features = df_accidents[['Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)',
#               'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit',
#               'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]

features = df_accidents[['Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit',
                         'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop']]

In [6]:
#Encoding text/categorical variables
le = LabelEncoder()
features['Weather_Condition']= le.fit_transform(features['Weather_Condition'])

C:\Users\forte\AppData\Local\Temp\ipykernel_16388\1052699683.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Weather_Condition']= le.fit_transform(features['Weather_Condition'])


In [7]:
#target
target = df_accidents['Severity']

In [8]:
#(80/20 split)
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [9]:
#Create a pipeline
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('feature_selection', SelectFromModel(LinearSVC(penalty="l1", dual=False), threshold='mean')),
    ('clf', SVC(max_iter=5000))  
])

In [10]:
#parameters for gridSearch
parameters = {
    'feature_selection__threshold': (None, 'mean'), 
    'clf__kernel': ('linear', 'rbf')
}

#Perform GridSearch
grid_search = GridSearchCV(pipe, param_grid=parameters, verbose=10)
grid_search.fit(features_train, target_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5; 1/4] START clf__kernel=linear, feature_selection__threshold=None.......


C:\Users\forte\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 1/5; 1/4] END clf__kernel=linear, feature_selection__threshold=None;, score=0.064 total time= 2.5min
[CV 2/5; 1/4] START clf__kernel=linear, feature_selection__threshold=None.......


C:\Users\forte\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 2/5; 1/4] END clf__kernel=linear, feature_selection__threshold=None;, score=0.131 total time= 2.2min
[CV 3/5; 1/4] START clf__kernel=linear, feature_selection__threshold=None.......


...

[CV 4/5; 4/4] END clf__kernel=rbf, feature_selection__threshold=mean;, score=0.452 total time= 4.5min
[CV 5/5; 4/4] START clf__kernel=rbf, feature_selection__threshold=mean..........


C:\Users\forte\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV 5/5; 4/4] END clf__kernel=rbf, feature_selection__threshold=mean;, score=0.803 total time= 4.5min


C:\Users\forte\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


GridSearchCV(estimator=Pipeline(steps=[('scale', StandardScaler()),
                                       ('feature_selection',
                                        SelectFromModel(estimator=LinearSVC(dual=False,
                                                                            penalty='l1'),
                                                        threshold='mean')),
                                       ('clf', SVC(max_iter=5000))]),
             param_grid={'clf__kernel': ('linear', 'rbf'),
                         'feature_selection__threshold': (None, 'mean')},
             verbose=10)

In [11]:
target_predicted = grid_search.predict(features_test)

In [12]:
#Evaluate the model
print("Default-Score des Klassifizierers: Accuracy=", grid_search.score(features_test, target_test), "\n")
print(classification_report(target_test, target_predicted))

Default-Score des Klassifizierers: Accuracy= 0.40694285714285716 

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       648
           2       0.82      0.44      0.58     56333
           3       0.13      0.27      0.18     11278
           4       0.03      0.24      0.05      1741

    accuracy                           0.41     70000
   macro avg       0.24      0.24      0.20     70000
weighted avg       0.68      0.41      0.49     70000



C:\Users\forte\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\forte\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\forte\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,